## Cleaning epoched data

In [ ]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

In [ ]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

In [ ]:
epochs.apply_baseline((None, 0))

In [ ]:
epochs.plot()

## Reject artifacts based on channel signal amplitude

In [ ]:
reject_criteria = dict(mag=3000e-15,     # 3000 fT
                       grad=3000e-13,    # 3000 fT/cm
                       eeg=150e-6,       # 150 µV
                       eog=200e-6)       # 200 µV

flat_criteria = dict(mag=1e-15,          # 1 fT
                     grad=1e-13,         # 1 fT/cm
                     eeg=1e-6)           # 1 µV

In [ ]:
epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

In [ ]:
epochs.plot_drop_log()

Quite a number of Epochs were dropped due to EOG artifacts.

In [ ]:
epochs['Visual'].plot_image()

In [ ]:
epochs.plot_sensors(ch_type='eeg')

In [ ]:
epochs['Visual'].plot_image(picks='EEG 060')

## SSP

### Let's see if we can retain most the epochs, but still get rid of the EOG artifact. And while we're at it, the ECG artifacts too 😉

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=0,
                                                           average=True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1,
                                                           n_eeg=1, average=True)

In [ ]:
eog_projs

In [ ]:
projs = eog_projs + ecg_projs
projs

In [ ]:
epochs.add_proj(projs)
epochs.plot()

In [ ]:
epochs_cleaned = epochs.copy().apply_proj()

epochs_cleaned['Visual'].plot_image()
epochs_cleaned['Visual'].plot_image(picks='EEG 060')

## ICA

First, start with the **raw data** again and apply a 1.0 Hz high-pass filter, which is advantegeous for ICA performance.

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1, h_freq=40)  # High-pass with 1. Hz cut-off is recommended for ICA

Read our epochs and extract which ones that were kept (**all** in our case, because we didn't apply any rejection procedure before saving the epochs in notebook 3; but this could be different in a real-world scenario, and you want to calculate ICA on the same set of epochs you're actually feeding into your analysis!)

In [ ]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection
epochs_selection

Only keep the subset of events that corresponds to the retained epochs.

In [ ]:
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

Create epochs for ICA. All parameters should match **exactly** the ones of the epochs we intend to clean.

In [ ]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True)

### Finally, fit ICA!

In [ ]:
epochs_ica.info

In [ ]:
n_components = 0.8  # Should normally be higher, like 0.999!!
method = 'picard'
max_iter = 100  # Should normally be higher, like 500 or even 1000!!
fit_params = dict(fastica_it=5)
random_state = 42

ica = mne.preprocessing.ICA(n_components=n_components,
    max_pca_components=300,
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)
ica.fit(epochs_ica)

In [ ]:
ica.plot_components(inst=epochs)

### Detect ECG and EOG patterns

In [ ]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(
    ecg_epochs, method='ctps')


eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(
    eog_epochs)

components_to_exclude = ecg_inds + eog_inds
ica.exclude = components_to_exclude

## Plot automated artifact detection scores

In [ ]:
ica.plot_scores(ecg_scores)

## Plot ICA sources

In [ ]:
ica.plot_sources(ecg_evoked)

## Plot overlay of original and cleaned data

In [ ]:
ica.plot_overlay(ecg_evoked)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Visualize artifact detection scores, ICA sources, and an overlay of original and cleaned data based on the EOG epochs.</li>
         <li>Visualize artifact detection scores, ICA sources, and an overlay of original and cleaned data based on the epochs we actually intend to analyze.</li>
    </ul>
</div>